In [ ]:
# run this cell to install requirements
!pip install -r examples_requirements.txt -i https://pypi.python.org/simple

# 3 Way Soccer

In this notebook we will use the ThreeWaySoccerEnv to check various betting strategies.
For data, we will use the free data provided by www.football-data.co.uk.
Specifically for this notebook we will use the English Premier League.

## Getting the data

The site keeps the data in CSV files, which we can easily load with pandas as such:

In [1]:
import pandas as pd
raw_odds_data = pd.concat([pd.read_csv('http://www.football-data.co.uk/mmz4281/{}{}/E0.csv'.format(i, i +1)) for i in range(10, 17)])
raw_odds_data['Date'] = pd.to_datetime(raw_odds_data['Date'], dayfirst=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Let's take a look at the first 5 rows of the data:

In [2]:
raw_odds_data.head(5)

,AC,AF,AR,AS,AST,AY,AwayTeam,B365A,B365D,B365H,...,SBH,SJA,SJD,SJH,VCA,VCD,VCH,WHA,WHD,WHH
0,7.0,15.0,0.0,12.0,2.0,2.0,West Ham,4.0,3.30,2.00,...,1.90,3.75,3.4,2.00,4.20,3.25,2.00,4.33,3.25,1.91
1,3.0,14.0,0.0,17.0,12.0,1.0,Everton,2.5,3.25,2.88,...,2.70,2.50,3.3,2.75,2.38,3.30,3.10,2.50,3.20,2.88
2,8.0,13.0,0.0,12.0,7.0,3.0,Fulham,3.4,3.30,2.20,...,2.10,3.00,3.4,2.30,3.40,3.30,2.20,3.50,3.20,2.15
3,1.0,10.0,0.0,10.0,4.0,0.0,West Brom,17.0,7.00,1.17,...,1.12,15.00,6.5,1.18,19.00,7.50,1.17,19.00,6.50,1.17
4,6.0,10.0,0.0,13.0,7.0,3.0,Birmingham,3.6,3.30,2.10,...,2.20,3.20,3.4,2.20,3.50,3.25,2.20,3.50,3.20,2.15


As you can see, we have alot of data to work with. We only need the team names, the odds and the result.
For the odds, we will use the maximum betting odds from betbrain.com (columns BbAvH, BbAvA and BbAvD for maximum home odds, maximum away odds and maximum draw odds.
As for the result, we will use the the FTR (Full Time Result) column, and change the values from (H, A, D) to (0, 1, 2).

In [3]:
odds_dataframe = raw_odds_data[['HomeTeam', 'AwayTeam', 'BbAvH', 'BbAvD', 'BbAvA']]
odds_dataframe.rename({'HomeTeam' :'home_team', 'AwayTeam': 'away_team', 'BbAvH': 'home',
                       'BbAvA': 'away', 'BbAvD': 'draw'}, axis='columns', inplace=True)
odds_dataframe['result'] = raw_odds_data['FTR'].map({'H': 0, 'A': 2, 'D': 1})
odds_dataframe.dropna(subset=['result'], inplace=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

Let's take a look at the first 5 rows of our odds dataframe:

In [4]:
odds_dataframe.head(5)

,home_team,away_team,home,draw,away,result
0,Aston Villa,West Ham,1.96,3.30,4.03,0.0
1,Blackburn,Everton,2.92,3.25,2.44,0.0
2,Bolton,Fulham,2.20,3.26,3.32,1.0
3,Chelsea,West Brom,1.16,6.90,17.47,0.0
4,Sunderland,Birmingham,2.18,3.25,3.39,1.0


Now we can use our custom environment and pass to it our odds dataframe:

In [5]:
from oddsgym.envs.soccer import ThreeWaySoccerOddsEnv
env = ThreeWaySoccerOddsEnv(odds_dataframe)
max_steps_limit = odds_dataframe.shape[0]
print(max_steps_limit)
print(env._results)

2660
[0. 0. 1. ... 2. 0. 2.]


Let's see what happens when we only bet on the home team:

In [6]:
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, info = env.step(2)
    print(info)
    if done:
        break

Home Team Aston Villa VS Away Team West Ham. Current balance at step 0: 10
{'action': 2, 'current_step': 0, 'balance': 10, 'odds': array([1.96, 3.3 , 4.03]), 'result': 0}
Home Team Blackburn VS Away Team Everton. Current balance at step 1: 9.0
{'action': 2, 'current_step': 1, 'balance': 9.0, 'odds': array([2.92, 3.25, 2.44]), 'result': 0}
Home Team Bolton VS Away Team Fulham. Current balance at step 2: 8.0
{'action': 2, 'current_step': 2, 'balance': 8.0, 'odds': array([2.2 , 3.26, 3.32]), 'result': 1}
Home Team Chelsea VS Away Team West Brom. Current balance at step 3: 10.26
{'action': 2, 'current_step': 3, 'balance': 10.26, 'odds': array([ 1.16,  6.9 , 17.47]), 'result': 0}
Home Team Sunderland VS Away Team Birmingham. Current balance at step 4: 9.26
{'action': 2, 'current_step': 4, 'balance': 9.26, 'odds': array([2.18, 3.25, 3.39]), 'result': 1}
Home Team Tottenham VS Away Team Man City. Current balance at step 5: 11.51
{'action': 2, 'current_step': 5, 'balance': 11.51, 'odds': array

{'action': 2, 'current_step': 473, 'balance': 20.54999999999999, 'odds': array([2.15, 3.27, 3.45]), 'result': 1}
Home Team Swansea VS Away Team Bolton. Current balance at step 474: 22.81999999999999
{'action': 2, 'current_step': 474, 'balance': 22.81999999999999, 'odds': array([2.09, 3.32, 3.54]), 'result': 0}
Home Team West Brom VS Away Team Liverpool. Current balance at step 475: 21.81999999999999
{'action': 2, 'current_step': 475, 'balance': 21.81999999999999, 'odds': array([3.77, 3.32, 2.02]), 'result': 2}
Home Team Wigan VS Away Team Fulham. Current balance at step 476: 20.81999999999999
{'action': 2, 'current_step': 476, 'balance': 20.81999999999999, 'odds': array([2.91, 3.23, 2.46]), 'result': 2}
Home Team Tottenham VS Away Team QPR. Current balance at step 477: 19.81999999999999
{'action': 2, 'current_step': 477, 'balance': 19.81999999999999, 'odds': array([1.37, 4.66, 8.56]), 'result': 0}
Home Team Stoke VS Away Team Newcastle. Current balance at step 478: 18.81999999999999
{'

We can see that after 60 games, we strike out. Let's try a random gambler:

In [7]:
from random import randint
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, info = env.step(env.action_space.sample())
    if done:
        break

Home Team Aston Villa VS Away Team West Ham. Current balance at step 0: 10
Home Team Blackburn VS Away Team Everton. Current balance at step 1: 10.0
Home Team Bolton VS Away Team Fulham. Current balance at step 2: 10.0
Home Team Chelsea VS Away Team West Brom. Current balance at step 3: 12.26
Home Team Sunderland VS Away Team Birmingham. Current balance at step 4: 12.42
Home Team Tottenham VS Away Team Man City. Current balance at step 5: 10.42
Home Team Wigan VS Away Team Blackpool. Current balance at step 6: 9.42
Home Team Wolves VS Away Team Stoke. Current balance at step 7: 11.92
Home Team Liverpool VS Away Team Arsenal. Current balance at step 8: 11.25
Home Team Man United VS Away Team Newcastle. Current balance at step 9: 12.49
Home Team Arsenal VS Away Team Blackpool. Current balance at step 10: 11.72
Home Team Birmingham VS Away Team Blackburn. Current balance at step 11: 10.88
Home Team Everton VS Away Team Wolves. Current balance at step 12: 10.88
Home Team Stoke VS Away Team

In [8]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
learning_env = DummyVecEnv([lambda: env])
model = PPO2(MlpPolicy, learning_env, verbose=1)
obs = learning_env.reset()
print(model.action_probability(obs))
model.learn(total_timesteps=2500)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[[0.12610477 0.12631975 0.12502472 0.12490653 0.1240982  0.12561397
  0.12325417 0.12467786]]
--------------------------------------
| approxkl           | 0.0002065824  |
| clipfrac           | 0.0           |
| explained_variance | 0.00463       |
| fps                | 330           |
| n_updates          | 1             |
| policy_entropy     | 2.0792215     |
| policy_loss        | -0.0013568744 |
| serial_timesteps   | 128           |
| time_elapsed       | 1.67e-06      |
| total_timesteps    | 128           |
| value_loss         | 21.217966     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00014403427 |
| clipfrac           | 0.0           |
| explained_variance | -0.00797      |
| fps

--------------------------------------
| approxkl           | 0.00025732163 |
| clipfrac           | 0.0           |
| explained_variance | 0.0112        |
| fps                | 1182          |
| n_updates          | 10            |
| policy_entropy     | 2.0690746     |
| policy_loss        | -0.0013330753 |
| serial_timesteps   | 1280          |
| time_elapsed       | 1.35          |
| total_timesteps    | 1280          |
| value_loss         | 27.744661     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00017272904 |
| clipfrac           | 0.0           |
| explained_variance | -0.00413      |
| fps                | 1240          |
| n_updates          | 11            |
| policy_entropy     | 2.0665662     |
| policy_loss        | -0.0036309855 |
| serial_timesteps   | 1408          |
| time_elapsed       | 1.46          |
| total_timesteps    | 1408          |
| value_loss         | 9.259583      |
-------------------------

In [10]:
test_odds_data = pd.read_csv('http://www.football-data.co.uk/mmz4281/1718/E0.csv')
test_odds_data['Date'] = pd.to_datetime(test_odds_data['Date'], dayfirst=True)
test_odds_dataframe = test_odds_data[['HomeTeam', 'AwayTeam', 'BbMxH', 'BbMxD', 'BbMxA']]
test_odds_dataframe.rename({'HomeTeam' :'home_team', 'AwayTeam': 'away_team', 'BbMxH': 'home',
                       'BbMxA': 'away', 'BbMxD': 'draw'}, axis='columns', inplace=True)
test_odds_dataframe['result'] = test_odds_data['FTR'].map({'H': 0, 'A': 2, 'D': 1})
test_odds_dataframe.dropna(subset=['result'], inplace=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [11]:
env = ThreeWaySoccerOddsEnv(test_odds_dataframe)
max_steps_limit = test_odds_dataframe.shape[0]
print(max_steps_limit)
print(env._results)

380
[0 2 2 2 0 1 1 0 0 2 2 2 0 0 0 0 2 0 2 1 2 2 1 0 0 1 0 0 1 1 0 0 2 2 0 2 1
 0 2 0 0 2 1 1 0 1 2 1 1 0 1 0 2 0 2 2 2 2 0 0 1 2 2 0 0 1 0 0 2 1 1 0 1 0
 0 0 0 1 1 1 2 0 0 0 0 0 2 2 2 0 0 2 1 0 0 2 2 1 0 0 0 2 2 1 2 2 0 0 0 0 0
 0 0 1 2 0 0 2 0 1 1 0 1 0 2 1 1 2 2 0 1 0 2 1 0 2 0 0 0 2 2 2 0 0 0 0 1 1
 1 0 0 1 0 2 0 0 0 1 2 1 0 0 2 1 0 2 2 2 0 1 0 1 0 2 0 2 2 2 2 0 1 0 2 1 1
 0 1 0 1 2 1 0 1 0 1 0 0 1 2 2 0 0 1 0 1 1 2 1 1 1 2 2 0 2 0 2 2 0 1 1 1 0
 2 1 0 1 0 0 0 0 0 2 2 1 0 0 0 1 1 0 2 0 1 2 0 0 1 1 1 0 0 0 0 1 1 0 2 1 1
 0 0 0 1 0 0 0 0 0 2 0 1 0 1 1 0 0 2 2 0 2 0 1 0 1 0 0 0 0 0 2 0 0 1 0 0 2
 2 0 2 2 0 2 0 2 2 2 2 0 0 1 2 0 0 2 1 1 1 2 2 2 2 1 0 1 0 0 0 0 2 1 2 2 0
 1 1 2 2 1 1 1 0 0 1 0 1 0 2 1 2 0 2 0 2 0 0 0 1 2 2 0 0 0 0 1 2 1 0 0 0 1
 2 0 2 0 0 0 2 2 0 0]


In [12]:
testing_env = env
obs = testing_env.reset()
for i in range(1, max_steps_limit):
    print(testing_env.render())
    print(obs)
    action, _states = model.predict(obs)
    print(testing_env._verbose_actions[action])
    obs, reward, done, _ = testing_env.step(action)
    print(reward)
    if done:
        break

Home Team Arsenal VS Away Team Leicester. Current balance at step 0: 10
[1.55 4.6  6.89]
()
0.0
Home Team Brighton VS Away Team Man City. Current balance at step 1: 10.0
[11.5   5.6   1.36]
()
0.0
Home Team Chelsea VS Away Team Burnley. Current balance at step 2: 10.0
[ 1.27  6.55 15.5 ]
('draw', 'away')
13.5
Home Team Crystal Palace VS Away Team Huddersfield. Current balance at step 3: 23.5
[1.86 3.65 5.11]
('draw', 'away')
3.1100000000000003
Home Team Everton VS Away Team Stoke. Current balance at step 4: 26.61
[1.71 3.85 6.  ]
('draw',)
-1.0
Home Team Southampton VS Away Team Swansea. Current balance at step 5: 25.61
[1.66 4.05 6.5 ]
('away',)
-1.0
Home Team Watford VS Away Team Liverpool. Current balance at step 6: 24.61
[6.5  4.3  1.65]
('home', 'away')
-2.0
Home Team West Brom VS Away Team Bournemouth. Current balance at step 7: 22.61
[2.5 3.3 3.3]
('draw',)
-1.0
Home Team Man United VS Away Team West Ham. Current balance at step 8: 21.61
[ 1.35  5.75 13.  ]
('home', 'draw', 'awa

()
0.0
Home Team Burnley VS Away Team Watford. Current balance at step 150: 13.519999999999992
[2.63 3.3  3.1 ]
('draw',)
-1.0
Home Team Crystal Palace VS Away Team Bournemouth. Current balance at step 151: 12.519999999999992
[2.1  3.53 4.16]
()
0.0
Home Team Huddersfield VS Away Team Brighton. Current balance at step 152: 12.519999999999992
[2.73 3.2  3.2 ]
('home', 'draw')
0.73
Home Team Newcastle VS Away Team Leicester. Current balance at step 153: 13.249999999999993
[2.7  3.35 2.95]
('home',)
-1.0
Home Team Swansea VS Away Team West Brom. Current balance at step 154: 12.249999999999993
[2.75 3.1  3.15]
('home', 'away')
0.75
Home Team Tottenham VS Away Team Stoke. Current balance at step 155: 12.999999999999993
[ 1.31  6.5  12.  ]
('home', 'away')
-0.69
Home Team West Ham VS Away Team Chelsea. Current balance at step 156: 12.309999999999993
[7.5  4.67 1.55]
('away',)
-1.0
Home Team Liverpool VS Away Team Everton. Current balance at step 157: 11.309999999999993
[ 1.4   6.29 12.  ]
('

[2.22 3.42 4.  ]
('draw',)
-1.0
Home Team Brighton VS Away Team Arsenal. Current balance at step 287: 39.76999999999999
[4.4  3.8  2.02]
('draw',)
-1.0
Home Team Man City VS Away Team Chelsea. Current balance at step 288: 38.76999999999999
[1.65 4.35 6.07]
('draw', 'away')
-2.0
Home Team Crystal Palace VS Away Team Man United. Current balance at step 289: 36.76999999999999
[7.   4.3  1.61]
()
0.0
Home Team Chelsea VS Away Team Crystal Palace. Current balance at step 290: 36.76999999999999
[ 1.33  6.   13.  ]
('home', 'away')
-0.6699999999999999
Home Team Everton VS Away Team Brighton. Current balance at step 291: 36.09999999999999
[2.26 3.3  3.89]
()
0.0
Home Team Huddersfield VS Away Team Swansea. Current balance at step 292: 36.09999999999999
[2.6  3.2  3.37]
()
0.0
Home Team Man United VS Away Team Liverpool. Current balance at step 293: 36.09999999999999
[2.81 3.4  2.82]
('away',)
-1.0
Home Team Newcastle VS Away Team Southampton. Current balance at step 294: 35.09999999999999
[2.8

In [13]:
testing_env.render()

'Home Team West Ham VS Away Team Everton. Current balance at step 379: 51.650000000000006'